# Object Detection

This script loads the object detection model and detects products, design features of displays and shelves. The detected objects are saved in an excel-file.

## Imports

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.builders import model_builder
from object_detection.utils import label_map_util
from object_detection_sabine import *

## Paths

In [ ]:
tf_wsp_path = os.environ.get('CAPSTONE_PATH')+'/TensorFlow/workspace/training_products_shelf/'
annotations_path = tf_wsp_path+'annotations'
config_path = tf_wsp_path+'models/my_ssd_resnet50_v1_fpn/pipeline.config'
ckpt_path = tf_wsp_path+'models/my_ssd_resnet50_v1_fpn'

## Load Model

First, we load the pipeline config and build a detection model.

In [ ]:
configs = config_util.get_configs_from_pipeline_file(config_path)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

Second, we restore the last checkpoint.

In [ ]:
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(ckpt_path, 'ckpt-26')).expect_partial()

## Label Map

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(annotations_path+'/label_map.pbtxt')

## Plot of Detected Objects

First, we save an image with one or all detected objects as image_np_with_detections.npy. Second, we plot the saved numpy array in the notebook shelf_analysis.plot.ipynb. We save the numpy array and plot in a separate notebook because the model_builder causes matplotlib not to plot in the same notebook.

In [ ]:
save_image_np_with_all_detections('images/2021/new_label/shop21.jpg', detection_model, category_index)
#save_image_np_with_one_detection('images/2021/new_label/shop1.jpg', detection_model, category_index, 0)

## TensorFlow to Excel: Sum of Objects

This part detects all objects in the images, and saves the number of displays, number of detected objects for each category as well as the number of shelves with the products of interest and the maximum number of products of interest on a shelf to an excel file.

First, I define a function that determines on which shelf a product is located.

In [ ]:
def shelf_number(product_box, a_shelf_df):
    # calculate intersection with each shelf box
    intersection_ls = []
    product_box_area = (product_box[1] - product_box[0])*(product_box[3] - product_box[2])
    for shelf_row, shelf in a_shelf_df.iterrows():
        # coordinates of intersection box
        xmin = max(product_box[0], shelf['xmin'])
        xmax = min(product_box[1], shelf['xmax'])
        ymin = max(product_box[2], shelf['ymin'])
        ymax = min(product_box[3], shelf['ymax'])

        intersection_ls.append(max(xmax - xmin, 0.0)*max(ymax - ymin, 0.0))

    if intersection_ls == []: # images without shelves
        return np.nan
    elif max(intersection_ls) == 0: # images with shelves but relevant products not on shelves
        return np.nan
    else:
        max_intersection = max(intersection_ls)
        if max_intersection > 0.7*product_box_area:
            return intersection_ls.index(max_intersection)
        else:
            return np.nan

In [ ]:
df = pd.DataFrame()
for jpg_file in os.listdir('images/2021/new_label'):
    file_path = os.path.join('images/2021/new_label', jpg_file)
    all_detections = detection_from_jpg(file_path, detection_model)
    detections = detection_selection(all_detections, 0.3) # selection of detections with a score above 0.3

    # number of displays (determined by presence of a certain design feature on top of shelf)
    ymax_ls = [obj[2] for obj in detections['detection_boxes']] # y_max of all detected objects (y=0 is upper left)
    ymax_main_design = [ymax_ls[idx] for idx in np.where(detections['detection_classes'] == 2)[0]] # y_max of this design feature
    ymax_avg = sum(ymax_ls) / len(ymax_ls)  # average y_max of all detected objects
    nr_displays = sum(ymax_main_design < ymax_avg)  # number of times the design feature shows up above the other detected objects

    # add how many products and design features (different categories) were detected
    values_to_add = {'displays': nr_displays}
    for idx, category in category_index.items():
        if idx in (1, 2): # first two classes are products
            values_to_add[category['name']] = sum(detections['detection_classes'] == (idx - 1)) # number of detections
        if idx in (3, 4, 5, 6): # other classes are design features showing up either on top or bottom of shelf
            ymax_class = [ymax_ls[idx] for idx in np.where(detections['detection_classes'] == (idx - 1))[0]]
            nr_on_top = sum(ymax_class < ymax_avg)
            nr_on_bottom = sum(ymax_class > ymax_avg)
            values_to_add['top_' + category['name']] = nr_on_top
            values_to_add['bottom_' + category['name']] = nr_on_bottom
        if idx == 7:
            values_to_add[category['name']] = sum(detections['detection_classes'] == (idx - 1)) # number of detections

    # add number of shelves and maximum number of products on a shelf
    image_df = pd.DataFrame(detections['detection_boxes'], columns=['ymin', 'xmin', 'ymax', 'xmax'])
    image_df['object'] = detections['detection_classes']
    product_df = image_df[image_df['object'].isin([0,1])] # dataframe with product bounding boxes as rows
    shelf_df = image_df[image_df['object'] == 6] # dataframe with shelf bounding boxes as rows
    product_df['shelf_nr'] = product_df[['xmin', 'xmax', 'ymin', 'ymax']].apply(lambda product: shelf_number(product, shelf_df), axis=1, result_type = 'expand')
    values_to_add['nr_stacked_shelves'] = len(product_df['shelf_nr'].dropna().unique())
    values_to_add['nr_products_without_shelf'] = product_df[product_df['shelf_nr'].isnull()]['object'].count()
    nr_products_on_shelves_df = product_df.groupby('shelf_nr')['object'].count()
    if product_df['shelf_nr'].isnull().all():
        values_to_add['max_nr_products_on_shelf'] = np.nan
    else:
        values_to_add['max_nr_products_on_shelf'] = max(product_df.groupby('shelf_nr')['object'].count())

    img_to_add = pd.Series(values_to_add, name=jpg_file[:-4])
    df = df.append(img_to_add)

df.head()

In [ ]:
df.to_excel('images/2021/tables/detected_objects.xlsx')